In [1]:
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize,sent_tokenize,PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords,state_union
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pickle
import time
from pprint import pprint
import inflect
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")


data = pd.read_csv("Amazon_Unlocked_Mobile.csv")
reviews = data['Reviews']

try:
    with open('master.pkl', 'rb') as f:
        master_list = pickle.load(f)
except:
    stop_words = set(stopwords.words("english"))
    master_list = []
    print(reviews[:2])

    ### Removing StopWords
    t1 = time.time()
    for ex in reviews:
        try:
            words = word_tokenize(ex)
        except Exception as e:
            pass
        filtered_sent = [w.lower() for w in words if not w in stop_words]
        master_list.append(filtered_sent)
    t2 = time.time()
    print("Time to remove stopwrods",t2-t1)
    print(master_list[:2])

    ###Removing Numbers
    num = inflect.engine() 
    t1 = time.time()
    for ex in master_list:
        for i in range(len(ex)):
            result = ''.join([j for j in ex[i] if not j.isdigit()])
            ex[i] = result
    t2 = time.time()
    print("Time to Remove Numbers",t2-t1)
    print(master_list[:2])



    ###Removing Punctuations
    p = RegexpTokenizer(r'\w+')
    t1 = time.time()
    temp = []
    for ex in master_list:
        try:
            st = ' '.join(ex)
            temp.append(("".join(j for j in st if j not in string.punctuation).split()))
        except Exception as e:
            pass

    t2 = time.time()
    master_list = [i for i in temp]
    print("Time to remove punctuations ",t2-t1)
    print(master_list[:2])


    # ### Stemming
    # ps = PorterStemmer()
    # t1 = time.time()
    # for ex in master_list:
    # 	for i in range(len(ex)):
    # 		try:
    # 			ex[i] = ps.stem(ex[i])
    # 		except Exception as e:
    # 			pass
    # t2 = time.time()
    # print("Time to Stem",t2-t1)
    # print(master_list[:2])


    # ## Lemmatizing
    # lemmatizer = WordNetLemmatizer()
    # t1 = time.time()
    # for ex in master_list:
    # 	for i in range(len(ex)):
    # 		try:
    # 			ex[i] = lemmatizer.lemmatize(ex[i])
    # 		except Exception as e:
    # 			pass
    # 			print(i)
    # t2 = time.time()
    # print("Time to lemmatize",t2-t1)
    # print(master_list[:2])

    import contractions

    ###Contraction
    t1 = time.time()
    for ex in master_list:
        for i in range(len(ex)):
            try:
                ex[i] = contractions.fix(ex[i])
            except Exception as e:
                pass
                print(i)
    t2 = time.time()
    print("Time to do Contraction ",t2-t1)
    print(master_list[:2])

    with open('master.pkl','wb') as f:
        pickle.dump(master_list, f)

/home/gaurav/Envs/SA/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gaurav/Envs/SA/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gaurav/Envs/SA/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gaurav/Envs/SA/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gaurav/Envs/SA/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data['Sentiment'] = 0
import sys
r = data['Rating']
s = []
for i in range(len(r)):
    if r[i] > 3:
        s.append(1)
    elif r[i] == 3:
        s.append(0)
    else:
        s.append(-1)
data['Sentiment'] = s

In [3]:
train = master_list[:331072]
test = master_list[331072:]
y_train = s[:331072]
y_test = s[331072:]


In [11]:
from gensim.models import Word2Vec

model.wv.save_word2vec_format('modelComplete.bin')
try:
    with open('model.pkl','rb') as f:
        model = pickle.load(f)
except:
    model = Word2Vec(master_list, size=100, window=10,min_count=2, workers=10)
    model.train(master_list, total_examples=len(master_list), epochs=10)
    with open('model.pkl','wb') as f:
        pickle.dump(model, f)

In [12]:
# from sklearn.decomposition import PCA
# from matplotlib import pyplot as plt

In [13]:
# X = model[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)

In [14]:
# %matplotlib inline
# plt.scatter(result[:, 0], result[:, 1])
# for i,word in enumerate(words):
#     plt.annotate(word, xy=(result[i, 0], result[i, 1]))
# plt.show()

In [15]:
w1 = 'good'
model.wv.most_similar(positive=w1)

/home/gaurav/Envs/SA/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('great', 0.7472882270812988),
 ('nice', 0.721825361251831),
 ('decent', 0.7105386853218079),
 ('excellent', 0.6371794939041138),
 ('ok', 0.6356492042541504),
 ('impressive', 0.593664824962616),
 ('bad', 0.5715458989143372),
 ('superb', 0.5674518346786499),
 ('awesome', 0.5635976195335388),
 ('goodthe', 0.5602595806121826)]

In [16]:
import numpy as np
import sys
words_in_consideration = set(model.wv.index2word)
def vecReview(sentence):
    vec = np.zeros(100 ,dtype='float64')
    for w in sentence:
        if w in words_in_consideration:
            vec += model[w]
    return vec

In [17]:
train_vec = []
for r in train:
    train_vec.append(vecReview(r))

/home/gaurav/Envs/SA/lib/python3.5/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [18]:
test_vec = []
for r in test:
    test_vec.append(vecReview(r))

/home/gaurav/Envs/SA/lib/python3.5/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from numpy.core.umath_tests import inner1d
try:
    with open('rfmodel.pkl','rb') as f:
        rfmodel = pickle.load(f)
except:
    rfmodel = RandomForestClassifier()
    rfmodel.fit(train_vec, y_train)
    with open('rfmodel.pkl','wb') as f:
        pickle.dump(rfmodel, f)
y_model = rfmodel.predict(test_vec)
print('Accuracy From RandomForest: ', 100 *accuracy_score(y_model,y_test))

/home/gaurav/Envs/SA/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Accuracy From RandomForest:  83.215735549971


In [21]:
print(y_model[:25])
print(y_test[:25])

[ 1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1 -1  1  1  1  1  1
 -1]
[1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1]


In [ ]:
from sklearn import svm
try:
    with open('svmmodel.pkl','rb') as f:
        SVMmodel = pickle.load(f)
except:
    SVMmodel = svm.SVC()
    SVMmodel.fit(train_vec, y_train)
    with open('svmmodel.pkl','wb') as f:
        pickle.dump(svmmodel, f)
y_model = SVMmodel.predict(test_vec)
print('Accuracy From SVM: ',100*accuracy_score(y_model,y_test))